In [1]:
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from openpyxl import Workbook, load_workbook
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [8]:
wbOrig = 'SEIR_Inputs.xlsm'

In [9]:
WBName = 'SEIR_Outputs.xlsx'
StartDate = datetime(2020, 6, 25)

In [12]:
wb = load_workbook(wbOrig)
current_datestamp = datetime.strftime(datetime.today(),'%d-%m-%Y')
wb_history = wbOrig
xl = pd.ExcelFile(wb_history)

In [13]:
df_R0 = xl.parse('R0matrix')
df_R0 = df_R0.drop(['Nr','Location'], axis = 1)
df_R0 = np.transpose(df_R0)
df_R0

,0,1,2,3,4,5,6,7,8
0,2.090490,2.197377,2.099157,3.468940,2.738283,2.185859,1.837189,2.200790,2.226440
1,0.182435,1.757902,0.316694,2.081364,0.140805,0.380866,0.103287,1.760632,1.028570
2,0.182435,1.406321,0.316694,0.832546,0.140805,0.380866,0.103287,1.232443,1.028570
3,0.127705,0.984425,0.221686,0.582782,0.098563,0.266606,0.072301,0.862710,0.719999
4,0.127705,0.984425,0.221686,0.582782,0.098563,0.266606,0.072301,0.862710,0.719999
5,0.153246,1.181310,0.266023,0.699338,0.118276,0.319928,0.086761,1.035252,0.863999
6,0.854543,0.898041,0.645733,0.871134,0.366524,0.467350,0.350766,0.825012,1.054336
7,1.555841,0.614772,1.025443,1.042930,0.614772,0.614772,0.614772,0.614772,1.244673
8,1.464495,0.632833,1.223241,1.608048,0.632833,0.632833,0.632833,0.632833,1.057418
9,1.373149,0.650895,1.421040,2.173165,0.650895,0.650895,0.650895,0.650895,0.870163


In [14]:
Scenarios = []

In [15]:
#This is our base case PROVINCIAL scenario
#R0_multiplier is a multiplier on the input R0s per sub-region
#WSName is the name of the sheet we will create to output results
#num_periods is periods over which we forecast. Len_period is days in 1 period. We forecast in blocks so we can introduce
#interventions/ varying R0's per period
#asymptomatic multiplier is redundant
#beta_multipler = mu;ltiplier on standard R0 for undetected class (assume undetecteds have lower R0)
#multipliers are where we want to vary the R0 per period we can put in a vector of multipliers.
#These will be applied to the standard R0 and undetected R0

#high_risk_S is the % of total population that is susceptible to COVID and uses standard parameters as defined above
#low_risk_S is the % of the population with a lower susceptibility to COVID
#low_risk_S_multipliers are multipliers to R0 for the low risk group (detected and undetected)

susceptible_perc = 1
R0_multiplier = 1
WSName = 'Province_Output'
num_periods = len(df_R0)
len_period = 30
asymptomatic_multiplier = 1
perc_a = 'NA'
perc_i = 'NA'
beta_a_multiplier = 0.5
multipliers = np.ones(num_periods)
dfName = 'Equations_Province'
# multipliers = [1,1,1,1,1,1,1,1,1,1,1,1,1,1]

high_risk_S = 0.10
low_risk_S = 0.60
low_risk_S_multipliers = [0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.12, 0.16]

Scenarios.append([susceptible_perc, R0_multiplier, WSName, num_periods, len_period, asymptomatic_multiplier, perc_a, \
                 perc_i, beta_a_multiplier, multipliers, dfName, high_risk_S, low_risk_S, low_risk_S_multipliers])

In [16]:
#This is our low case PROVINCIAL scenario
#R0_multiplier is a multiplier on the input R0s per sub-region
#WSName is the name of the sheet we will create to output results
#num_periods is periods over which we forecast. Len_period is days in 1 period. We forecast in blocks so we can introduce
#interventions/ varying R0's per period
#asymptomatic multiplier is redundant
#beta_multipler = mu;ltiplier on standard R0 for undetected class (assume undetecteds have lower R0)
#multipliers are where we want to vary the R0 per period we can put in a vector of multipliers.
#These will be applied to the standard R0 and undetected R0

#high_risk_S is the % of total population that is susceptible to COVID and uses standard parameters as defined above
#low_risk_S is the % of the population with a lower susceptibility to COVID
#low_risk_S_multipliers are multipliers to R0 for the low risk group (detected and undetected)

susceptible_perc = 1
R0_multiplier = 1
WSName = 'Province_Output_low'
num_periods = len(df_R0)
len_period = 30
asymptomatic_multiplier = 1
perc_a = 'NA'
perc_i = 'NA'
beta_a_multiplier = 0.5
multipliers = np.ones(num_periods)
dfName = 'Equations_Province'
multipliers = [1,1,0.9,0.8,0.8,0.9,1,1,1,1,1,1]


high_risk_S = 0.05
low_risk_S = 0.55
# low_risk_S_multipliers = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.11, 0.11, 0.11, 0.11, 0.11]
low_risk_S_multipliers = [0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.12, 0.16]

Scenarios.append([susceptible_perc, R0_multiplier, WSName, num_periods, len_period, asymptomatic_multiplier, perc_a, \
                 perc_i, beta_a_multiplier, multipliers, dfName, high_risk_S, low_risk_S, low_risk_S_multipliers])

In [17]:
#This is our high case PROVINCIAL scenario
#R0_multiplier is a multiplier on the input R0s per sub-region
#WSName is the name of the sheet we will create to output results
#num_periods is periods over which we forecast. Len_period is days in 1 period. We forecast in blocks so we can introduce
#interventions/ varying R0's per period
#asymptomatic multiplier is redundant
#beta_multipler = mu;ltiplier on standard R0 for undetected class (assume undetecteds have lower R0)
#multipliers are where we want to vary the R0 per period we can put in a vector of multipliers.
#These will be applied to the standard R0 and undetected R0

#high_risk_S is the % of total population that is susceptible to COVID and uses standard parameters as defined above
#low_risk_S is the % of the population with a lower susceptibility to COVID
#low_risk_S_multipliers are multipliers to R0 for the low risk group (detected and undetected)

susceptible_perc = 1
R0_multiplier = 1
WSName = 'Province_Output_high'
num_periods = len(df_R0)
len_period = 30
asymptomatic_multiplier = 1
perc_a = 'NA'
perc_i = 'NA'
beta_a_multiplier = 0.5
# multipliers = np.ones(num_periods)
dfName = 'Equations_Province'
multipliers = [1,1.1,1.1,1.1,1.1,1,1,1,1,1,1,1]


high_risk_S = 0.15
low_risk_S = 0.65
low_risk_S_multipliers = [0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.12, 0.16]

Scenarios.append([susceptible_perc, R0_multiplier, WSName, num_periods, len_period, asymptomatic_multiplier, perc_a, \
                 perc_i, beta_a_multiplier, multipliers, dfName, high_risk_S, low_risk_S, low_risk_S_multipliers])

In [18]:
# #CAN IGNORE THIS SCENARIO -OLD

# susceptible_perc = 1
# R0_multiplier = 1
# WSName = 'Province_Output_low_flatR0'
# num_periods = 12
# len_period = 30
# asymptomatic_multiplier = 1
# perc_a = 'NA'
# perc_i = 'NA'
# beta_a_multiplier = 0.5
# multipliers = np.ones(num_periods)
# dfName = 'Equations_Province'
# # multipliers = [1,1,0.9,0.8,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7]

# high_risk_S = 0.1
# low_risk_S = 0.35
# low_risk_S_multipliers = [0.00, 0.01, 0.025, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.1, 0.1, 0.1, 0.1]

# Scenarios.append([susceptible_perc, R0_multiplier, WSName, num_periods, len_period, asymptomatic_multiplier, perc_a, \
#                  perc_i, beta_a_multiplier, multipliers, dfName, high_risk_S, low_risk_S, low_risk_S_multipliers])

In [19]:
# #This is our base case DISTRICT scenario

# susceptible_perc = 1
# R0_multiplier = 1
# WSName = 'District_Output'
# num_periods = 12
# len_period = 30
# asymptomatic_multiplier = 1
# perc_a = 'NA'
# perc_i = 'NA'
# beta_a_multiplier = 0.5
# multipliers = np.ones(num_periods)
# dfName = 'Equations_District'
# # multipliers = [1,1,0.9,0.8,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7]

# high_risk_S = 0.1
# low_risk_S = 0.35
# low_risk_S_multipliers = [0.00, 0.01, 0.025, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.1, 0.1, 0.1, 0.1]

# Scenarios.append([susceptible_perc, R0_multiplier, WSName, num_periods, len_period, asymptomatic_multiplier, perc_a, \
#                  perc_i, beta_a_multiplier, multipliers, dfName, high_risk_S, low_risk_S, low_risk_S_multipliers])

In [20]:
try:
    del wb['Province_Output']
except:
    pass
try:
    del wb['Province_Output_low']
except:
    pass
try:
    del wb['Province_Output_high']
except:
    pass
try:
    del wb['Province_Output_low_flatR0']
except:
    pass
try:
    del wb['District_Output']
except:
    pass
    
# del wb['Muni_Output']

In [21]:
df_orig = 0
S = 0
E = 0
I = 0
I_a = 0
I_i = 0
R = 0
D = 0
y = 0

In [22]:
#This is the system of ordinary differential equations we interpolate over

def deriv(y, t, beta_a, beta_i, perc_a, perc_i, gamma, alpha, mu, i_move, col):
    
    su = np.ones_like(mu)-mu
    
    y_new = np.reshape(y,(int(len(y)/col),col))
    
    S = y_new[0]
    E = y_new[1]
    I_i = y_new[2]
    I_a = y_new[3]
    R = y_new[4]
    D = y_new[5]
    
    i_move_from_S = i_move
    i_move_to_S = np.transpose(i_move)
    Popn_home = S + E + I + R
    Popn_home_inv = 1 / Popn_home
    Popn_away = np.transpose(Popn_home)
    Popn_away_inv = 1 / Popn_away
    i_stayhome = np.diagonal(i_move)
    
    dSdt =  - S*(I_i*Popn_away_inv*beta_i).dot(i_move_from_S) \
            - S*(I_i*Popn_home_inv*beta_i).dot(i_move_to_S) \
            + S*(I_i*Popn_home_inv*beta_i*i_stayhome) \
            - S*(I_a*Popn_away_inv*beta_a).dot(i_move_from_S) \
            - S*(I_a*Popn_home_inv*beta_a).dot(i_move_to_S) \
            + S*(I_a*Popn_home_inv*beta_a*i_stayhome)
    
    dEdt =  + S*(I_i*Popn_away_inv*beta_i).dot(i_move_from_S) \
            + S*(I_i*Popn_home_inv*beta_i).dot(i_move_to_S) \
            - S*(I_i*Popn_home_inv*beta_i*i_stayhome) \
            + S*(I_a*Popn_away_inv*beta_a).dot(i_move_from_S) \
            + S*(I_a*Popn_home_inv*beta_a).dot(i_move_to_S) \
            - S*(I_a*Popn_home_inv*beta_a*i_stayhome) \
            - alpha*E
    
    dI_idt =  perc_i*alpha*E - gamma * I_i
    dI_adt =  perc_a*alpha*E - gamma * I_a
    dRdt =  gamma * I_i * su + gamma*I_a * su
    dDdt =  gamma * I_i * mu + gamma * I_a * mu

    y_new = np.array([dSdt,dEdt,dI_idt, dI_adt, dRdt,dDdt])
    y_new = np.reshape(y_new,(1,len(dSdt)+len(dEdt)+len(dI_idt)+len(dI_adt)+len(dRdt)+len(dDdt)))
    y_new = np.squeeze(y_new)

    return y_new

In [23]:
#here we run the ODE integration for each scenario, output into a spreadsheet and save

for item in Scenarios:
    
    susceptible_perc = item[0]
    R0_multiplier = item[1]
    WSName = item[2]
    num_periods = item[3]
    len_period = item[4]
    asymptomatic_multiplier = item[5]
    beta_a_multiplier = item[8]
    multipliers = item[9]
    dfName = item[10]
    high_risk_S = item[11]
    low_risk_S = item[12]
    low_risk_S_multipliers = item[13]
    
    df_orig = xl.parse(dfName)
    df_orig.fillna(0)
    
    df_scen = df_R0

    df = df_orig.copy()
    
    S_orig = df['N'] - df['I'] - df['U'] - df['R'] - df['D'] - df['E']

    df['alpha'] = 1/df['Tinc']
    df['gamma'] = 1/df['Tinf']
    beta_i = multipliers*np.transpose(R0_multiplier*df_scen/df['Tinf'])
    beta_a = beta_a_multiplier*beta_i
    df['I_i'] = df['I']
    df['I_a'] = df['U']
    df['S'] = susceptible_perc*(df['N'] - df['I'] - df['U'] - df['R'] - df['D'] - df['E'])
    df['perc_u']
    df.drop(['R0', 'Tinc', 'Tinf','I','U','beta'], axis=1,inplace=True)
    
    perc_a = df['perc_u']
    temp_len = len(perc_a)
    perc_i = np.ones(temp_len) - perc_a
    Loc = df['Location']
    S0 = (high_risk_S + low_risk_S*low_risk_S_multipliers[0]) * df['S']
    E0 = df['E']
    I_i0 = df['I_i']
    I_a0 = df['I_a']
    R0 = df['R']
    D0 = df['D']
#     beta_i = df['beta_i']
#     beta_a = df['beta_a']
    alpha = df['alpha']
    gamma = df['gamma']
    mu = df['mu']
    i_move = df
    i_move.drop(['Nr','Location','N','R','I_i','I_a','E','D','S','mu','alpha','gamma','perc_u'], axis=1,inplace=True)
    i_move = i_move.to_numpy()
    i_move = np.transpose(i_move)

    Forecast_days = len_period
    t = np.linspace(0, Forecast_days, Forecast_days)

    y0 = np.array([S0,E0,I_i0,I_a0,R0,D0])
    col = len(S0)
    y0 = np.reshape(y0,(1,len(S0)+len(E0)+len(I_i0)+len(I_a0)+len(R0)+len(D0)))
    y0 = np.squeeze(y0)

    S_tot = []
    E_tot = []
    I_itot = []
    I_atot = []
    R_tot = []
    D_tot = []
    S_tot_temp = []
    E_tot_temp = []
    I_itot_temp = []
    I_atot_temp = []
    R_tot_temp = []
    D_tot_temp = []
#     StartDate = datetime.today()
    DateRng_Tot = []
    DateRngB_Tot = []

    daterng = np.array([StartDate + relativedelta(days = i) for i in range(0,num_periods*Forecast_days)])
    daterngB = np.array([datetime.strftime(StartDate + relativedelta(days = i),'%d-%b-%y') for i in range(0,num_periods*Forecast_days)])

#     beta_i_all = [i*beta_i for i in multipliers]
#     beta_a_all = [beta_a_multiplier*i*beta_i for i in multipliers]

    for i in range(0, num_periods):
        
        ret = odeint(deriv, y0, t, args=(beta_a[i], beta_i[i], perc_a, perc_i, gamma, alpha, mu, i_move, col))
        
        ret_ar = np.array(ret)

        col_size = int(len(ret_ar[1])/col)
        S_new = ret_ar[:,0:col]
        E_new = ret_ar[:,col:2*col]
        I_inew = ret_ar[:,2*col:3*col]
        I_anew = ret_ar[:,3*col:4*col]
        R_new = ret_ar[:,4*col:5*col]
        D_new = ret_ar[:,5*col:]
        
        if i == 0:

            for j in range(0,len(S_new)):
                S_tot_temp = []
                for k in range(0,len(S_new[j])):
                    S_tot_temp.append(S_new[j][k])
                S_tot.append(S_tot_temp)

            for j in range(0,len(E_new)):
                E_tot_temp = []
                for k in range(0,len(E_new[j])):
                    E_tot_temp.append(E_new[j][k])
                E_tot.append(E_tot_temp)

            for j in range(0,len(I_inew)):
                I_itot_temp = []
                for k in range(0,len(I_inew[j])):
                    I_itot_temp.append(I_inew[j][k])
                I_itot.append(I_itot_temp)

            for j in range(0,len(I_anew)):
                I_atot_temp = []
                for k in range(0,len(I_anew[j])):
                    I_atot_temp.append(I_anew[j][k])
                I_atot.append(I_atot_temp)

            for j in range(0,len(R_new)):
                R_tot_temp = []
                for k in range(0,len(R_new[j])):
                    R_tot_temp.append(R_new[j][k])
                R_tot.append(R_tot_temp)

            for j in range(0,len(D_new)):
                D_tot_temp = []
                for k in range(0,len(D_new[j])):
                    D_tot_temp.append(D_new[j][k])
                D_tot.append(D_tot_temp)
        else:
            
            for j in range(1,len(S_new)):
                S_tot_temp = []
                for k in range(0,len(S_new[j])):
                    S_tot_temp.append(S_new[j][k])
                S_tot.append(S_tot_temp)

            for j in range(1,len(E_new)):
                E_tot_temp = []
                for k in range(0,len(E_new[j])):
                    E_tot_temp.append(E_new[j][k])
                E_tot.append(E_tot_temp)

            for j in range(1,len(I_inew)):
                I_itot_temp = []
                for k in range(0,len(I_inew[j])):
                    I_itot_temp.append(I_inew[j][k])
                I_itot.append(I_itot_temp)

            for j in range(1,len(I_anew)):
                I_atot_temp = []
                for k in range(0,len(I_anew[j])):
                    I_atot_temp.append(I_anew[j][k])
                I_atot.append(I_atot_temp)

            for j in range(1,len(R_new)):
                R_tot_temp = []
                for k in range(0,len(R_new[j])):
                    R_tot_temp.append(R_new[j][k])
                R_tot.append(R_tot_temp)

            for j in range(1,len(D_new)):
                D_tot_temp = []
                for k in range(0,len(D_new[j])):
                    D_tot_temp.append(D_new[j][k])
                D_tot.append(D_tot_temp)
        
        if i == num_periods - 1:
            S_update = S_new[len(S_new)-1] + low_risk_S*low_risk_S_multipliers[i] * S_orig
        else:
            S_update = S_new[len(S_new)-1] + low_risk_S*low_risk_S_multipliers[i+1] * S_orig
        E_update = E_new[len(S_new)-1]
        I_iupdate = I_inew[len(S_new)-1]
        I_aupdate = I_anew[len(S_new)-1]
        R_update = R_new[len(S_new)-1]
        D_update = D_new[len(S_new)-1]

        y0 = np.array([S_update,E_update,I_iupdate,I_aupdate,R_update,D_update])
        y0 = np.reshape(y0,(1,len(S_update)+len(E_update)+len(I_iupdate)+len(I_aupdate)+len(R_update)+len(D_update)))
        y0 = np.squeeze(y0)

    ws = wb.create_sheet(WSName, 0)

    j = 0
    while j < 6:
        for i in range(0,len(Loc)):
            if j == 0:
                ws.cell(row = 1, column = 2+ j*len(Loc) + i).value = 'Susceptible'
            elif j == 1:
                ws.cell(row = 1, column = 2+ j*len(Loc) + i).value = 'Exposed'
            elif j == 2:
                ws.cell(row = 1, column = 2+ j*len(Loc) + i).value = 'Infected_i'
            elif j == 3:
                ws.cell(row = 1, column = 2+ j*len(Loc) + i).value = 'Infected_a'
            elif j == 4:
                ws.cell(row = 1, column = 2+ j*len(Loc) + i).value = 'Recovered'
            elif j == 5:
                ws.cell(row = 1, column = 2+ j*len(Loc) + i).value = 'Dead'
            ws.cell(row = 2, column = 2+ j*len(Loc) + i).value = Loc[i]
            ws.cell(row = 3, column = 2+ j*len(Loc) + i).value = ws.cell(row = 1, column = 2+ j*len(Loc) + i).value + '_' + Loc[i]
        j += 1

    for i in range(0,len(daterng)):
        ws.cell(row=4+i, column=1).value = daterngB[i]

    for i in range(0,col):
        for j in range(0, len(S_tot)):
            ws.cell(row = 4 + j, column = 2 + i).value = round(S_tot[j][i])

    for i in range(0,col):
        for j in range(0, len(E_tot)):
            ws.cell(row = 4 + j, column = 2 + len(Loc) + i).value = round(E_tot[j][i])

    for i in range(0,col):
        for j in range(0, len(I_itot)):
            ws.cell(row = 4 + j, column = 2 + 2*len(Loc) + i).value = round(I_itot[j][i])

    for i in range(0,col):
        for j in range(0, len(I_atot)):
            ws.cell(row = 4 + j, column = 2 + 3*len(Loc) + i).value = round(I_atot[j][i])

    for i in range(0,col):
        for j in range(0, len(R_tot)):
            ws.cell(row = 4 + j, column = 2 + 4*len(Loc) + i).value = round(R_tot[j][i])

    for i in range(0,col):
        for j in range(0, len(D_tot)):
            ws.cell(row = 4 + j, column = 2 + 5*len(Loc) + i).value = round(D_tot[j][i])

In [24]:
wb.save(WBName)